# Book 4 - Combinando Modelos de Machine Learning

Este notebook serviu como registro prático e teórico no meu aprendizado de Machine Learning.

`Enriqueci este notebook com anotações adicionais e aplicações práticas tornando-o uma referência valiosa para consultas e implementações em futuros projetos reais.`

Espero que este material inspire outros a explorar ainda mais o fascinante mundo do Machine Learning. 

No notebook presente tem todos os topicos dos notebook anteriores, porém sendo acrescentado e aprofundado com anotações dos seguintes tópicos:

- **Votação**
- **Bagging**
- **Boosting**
- **Stacking**


Compartilhar conhecimento é uma alegria—viva ao aprendizado contínuo, boa pratica e bons estudo a quem estiver lendo, abraços!

# Funções, bibliotecas e Dataframe ficticios

In [84]:
RANDOM_STATE = 3141592

In [85]:
import matplotlib.pyplot as plt
plt.style.use('dark_background')
%matplotlib inline

# Manipulação e Tratamento de dados
import openpyxl
import pandas as pd
import numpy as np
from numpy import NaN

#ignorando Warning inuteis
import warnings 
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [86]:
import requests

# CARREGANDO BASE
arquivo = 'fake_database'
url = "https://raw.githubusercontent.com/GabrielGabes/functions_gsa/main/" + arquivo + ".py"
response = requests.get(url)
code = response.text
exec(code)
df = fake_database2()
display(df.head())

arquivo = 'funcoes_estatisticas'
url = "https://raw.githubusercontent.com/GabrielGabes/functions_gsa/main/" + arquivo + ".py"
response = requests.get(url)
code = response.text
exec(code)
print('TUDO OK')

,x_num0,x_num1,x_num2,x_num3,x_num4,x_num5,x_num6,x_num7,x_num8,x_num9,...,x_bin0,x_bin1,x_bin2,x_bin3,x_bin4,x_cat0,x_cat1,x_cat2,x_cat_0,x_cat_1
0,4.875997,-1.491729,2.491069,1.874948,-0.000381,1.478758,3.972563,0.230329,5.149503,0.595726,...,sim,não,sim,não,sim,A,B,A,C,D
1,3.750108,-0.510656,-0.462908,0.928715,0.496968,-0.995687,2.219878,-1.951839,0.335021,-0.071709,...,não,não,não,sim,não,B,B,C,C,B
2,1.915908,2.793605,2.989653,1.743696,-0.213394,-0.274895,0.106642,1.605156,2.169961,-1.126328,...,sim,sim,sim,não,sim,C,A,C,B,C
3,3.575351,2.779022,4.928382,3.046386,-1.832086,-2.961492,1.171947,1.044997,3.281876,-1.733729,...,não,não,não,sim,não,B,A,A,D,D
4,4.812294,0.316227,0.704474,0.908321,0.050424,-2.080064,2.597434,-1.643092,-0.138313,-1.498914,...,sim,não,não,não,não,A,B,A,D,B


TUDO OK


In [87]:
# Variavel Dependente
var_dep = 'y'
y = df[var_dep]
display(y.head())

x = df.drop('y', axis=1)
display(x.head())

0    0
1    1
2    0
3    1
4    1
Name: y, dtype: object

,x_num0,x_num1,x_num2,x_num3,x_num4,x_num5,x_num6,x_num7,x_num8,x_num9,x_bin0,x_bin1,x_bin2,x_bin3,x_bin4,x_cat0,x_cat1,x_cat2,x_cat_0,x_cat_1
0,4.875997,-1.491729,2.491069,1.874948,-0.000381,1.478758,3.972563,0.230329,5.149503,0.595726,sim,não,sim,não,sim,A,B,A,C,D
1,3.750108,-0.510656,-0.462908,0.928715,0.496968,-0.995687,2.219878,-1.951839,0.335021,-0.071709,não,não,não,sim,não,B,B,C,C,B
2,1.915908,2.793605,2.989653,1.743696,-0.213394,-0.274895,0.106642,1.605156,2.169961,-1.126328,sim,sim,sim,não,sim,C,A,C,B,C
3,3.575351,2.779022,4.928382,3.046386,-1.832086,-2.961492,1.171947,1.044997,3.281876,-1.733729,não,não,não,sim,não,B,A,A,D,D
4,4.812294,0.316227,0.704474,0.908321,0.050424,-2.080064,2.597434,-1.643092,-0.138313,-1.498914,sim,não,não,não,não,A,B,A,D,B


In [88]:
# DUMMYRIZAÇÃO
colunas_categoricas = []
colunas_binarias = []
colunas_mais3_categorias = []

for coluna in x.columns:
    if df[coluna].dtype == 'O':
        categorias = x[coluna].unique()
        if len(categorias) == 2:
            print('2 niveis:', coluna, '=>', categorias)
            colunas_categoricas.append(coluna)
            colunas_binarias.append(coluna)
        else:
            print('3 niveis:', coluna, '=>', categorias)
            colunas_categoricas.append(coluna)
            colunas_mais3_categorias.append(coluna)

############################################################################################
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder #transformando colunas com 2 categorias em 0 e 1

coluna = x.columns
one_hot = make_column_transformer((
    OneHotEncoder(drop='if_binary'), #caso a coluna tenha apenas 2 categorias 
    colunas_categoricas), #passando quais são essas colunas
    remainder = 'passthrough', sparse_threshold=0) #oque deve ser feito com as outras

#Aplicando transformação
x = one_hot.fit_transform(x)

#Os novos nomes das colunas #'onehotencoder=transformadas; 'remainder'=não transformadas
novos_nomes_colunas = one_hot.get_feature_names_out(coluna)

x = pd.DataFrame(x, columns = novos_nomes_colunas) #alterando de volta
x_columns = x.columns.tolist() 

############################################################################################
# NORMALIZAÇÃO
from sklearn.preprocessing import MinMaxScaler
normalizacao = MinMaxScaler()
x = normalizacao.fit_transform(x)

#df['Close_padronizada'] = (df[coluna] - df[coluna].mean()) / df[coluna].std()
#df['Close_normalizada'] = (df[coluna] - df[coluna].min()) / (df[coluna].max() - df[coluna].min())

############################################################################################
# DEFININDO A VARIAVEL DEPENDENTE
from sklearn.preprocessing import LabelEncoder
y = LabelEncoder().fit_transform(y)

############################################################################################
x_inteiro = x
y_inteiro = y

# DIVIDINDO BASE EM TREINO E TESTE
from sklearn.model_selection import train_test_split
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, 
                                                    stratify = y, #para manter a proporção da Var Dep nos splits
                                                    random_state = 5) #raiz da aleatoridade
# test_size = 0.25 #porcentagem que ira ser separado para testes

print(x_treino.shape, x_teste.shape)
print(y_treino.shape, y_teste.shape)

2 niveis: x_bin0 => ['sim' 'não']
2 niveis: x_bin1 => ['não' 'sim']
2 niveis: x_bin2 => ['sim' 'não']
2 niveis: x_bin3 => ['não' 'sim']
2 niveis: x_bin4 => ['sim' 'não']
3 niveis: x_cat0 => ['A' 'B' 'C']
3 niveis: x_cat1 => ['B' 'A' 'C']
3 niveis: x_cat2 => ['A' 'C' 'B']
3 niveis: x_cat_0 => ['C' 'B' 'D' 'A']
3 niveis: x_cat_1 => ['D' 'B' 'C' 'A']
(750, 32) (250, 32)
(750,) (250,)


In [89]:
# Função para avaliação de modelos exibindo metricas de avaliação
import requests
arquivo = 'ML_supervised_learning'
url = "https://raw.githubusercontent.com/GabrielGabes/functions_gsa/main/" + arquivo + ".py"
response = requests.get(url)
code = response.text
exec(code)

# =======================================

# VotingClassifier - Votação

`Combinar tres modelos para tomada de uma decisão.`

VotingClassifier é uma técnica de aprendizado de máquina que combina diferentes modelos de classificação para produzir uma previsão mais precisa e robusta. Imagine que você tem vários modelos de classificação, cada um com suas próprias forças e fraquezas. Em vez de escolher apenas um modelo, o VotingClassifier permite que você tire o melhor de todos eles.

A ideia por trás do VotingClassifier é simples: cada modelo de classificação faz sua previsão e, em seguida, o VotingClassifier conta os votos de cada modelo. A classe que receber a maioria dos votos é a previsão final. Isso é conhecido como “votação majoritária” ou “votação rígida”.

O VotingClassifier é uma técnica poderosa porque permite combinar modelos que são bons em capturar diferentes tipos de padrões nos dados. Por exemplo, um modelo pode ser bom em capturar tendências lineares, enquanto outro pode ser bom em capturar interações não lineares. Ao combiná-los com o VotingClassifier, você pode capturar ambos os tipos de padrões.

Abaixo estamos criando alguns modelos, avaliando antes de seguirmos para combina-los com voting

In [90]:
import plotly.express as px

In [91]:
# VALIDAÇÃO CRUZADA ESTRATIFICADA
from sklearn.model_selection import cross_validate, StratifiedKFold
skf = StratifiedKFold(n_splits = 10, 
                      shuffle = True, # embaralhamento dos dados
                      random_state = 5)

# INSTANCIANDO E VALIDANDO 3 MODELOS ISOLADAMENTE
#####################################################################
# MODELO 1 ##########################################################
from sklearn.tree import DecisionTreeClassifier
modelo1 = DecisionTreeClassifier(random_state=5, max_depth = 3)
cv_resultados1 = cross_validate(modelo1, x_treino, y_treino, cv = skf)
print(cv_resultados1['test_score'].mean())

# MODELO 2 ##########################################################
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
#modelo2 = LogisticRegression(random_state=1)
modelo2 = make_pipeline(StandardScaler(), LogisticRegression(random_state=1))
cv_resultados2 = cross_validate(modelo2, x_treino, y_treino, cv = skf)
print(cv_resultados2['test_score'].mean())

# MODELO 3 ##########################################################
from sklearn.naive_bayes import GaussianNB
modelo3 = GaussianNB()
cv_resultados3 = cross_validate(modelo3, x_treino, y_treino, cv = skf)
print(cv_resultados3['test_score'].mean())

#####################################################################
df1 = pd.DataFrame(cv_resultados1)
df1['Modelo'] = 'Decision Tree'
df2 = pd.DataFrame(cv_resultados2)
df2['Modelo'] = 'Logistic Regression'
df3 = pd.DataFrame(cv_resultados3)
df3['Modelo'] = 'Naive Bayes'
dados_combinados = pd.concat([df1, df2, df3])
num_cat_hip(dados_combinados, 'Modelo', 'test_score')

0.808
0.9226666666666666
0.9
P-value (Anova): < 0.001


Avaliar se o modelo é bom, se vale a pena prosseguir com ele

In [92]:
# avaliando modelo isoladamente
modelo1.fit(x_treino, y_treino)
previsoes =  modelo1.predict(x_teste)
avaliar_modelo(y_teste, previsoes, 'Decision Tree')

modelo2.fit(x_treino, y_treino)
previsoes =  modelo2.predict(x_teste)
avaliar_modelo(y_teste, previsoes, 'Logistic Regression')

modelo3.fit(x_treino, y_treino)
previsoes =  modelo3.predict(x_teste)
avaliar_modelo(y_teste, previsoes, 'Naive Bayes')

**********************************************************************
Relatório de Classificação para o Conjunto de Decision Tree :

              precision    recall  f1-score   support

           0       0.72      0.84      0.77       124
           1       0.81      0.67      0.74       126

    accuracy                           0.76       250
   macro avg       0.76      0.76      0.75       250
weighted avg       0.76      0.76      0.75       250

**************************************************
Matriz de Confusão para o Conjunto de Decision Tree :

[[104  20]
 [ 41  85]]
**********************************************************************
**********************************************************************
Relatório de Classificação para o Conjunto de Logistic Regression :

              precision    recall  f1-score   support

           0       0.88      0.92      0.90       124
           1       0.92      0.88      0.90       126

    accuracy                       

In [93]:
modelo1 = DecisionTreeClassifier(random_state=5, max_depth = 3)
modelo2 = LogisticRegression(random_state=1)
modelo3 = GaussianNB()

nome_modelos = ['Árvores','Logística','Naive Bayes']
pipelines = []
dados_combinados = pd.DataFrame()

for modelo, nome in zip([modelo1,modelo2,modelo3],nome_modelos):

  pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Normalização dos dados
    ('model', modelo)
  ])
  pipelines.append(pipeline)

  validacao = cross_validate(pipeline, x_treino, y_treino, cv = skf)
  #print(validacao['test_score'].mean())
  df = pd.DataFrame(validacao)
  df['Modelo'] = nome
  dados_combinados = pd.concat([dados_combinados, df])

num_cat_hip(dados_combinados, 'Modelo', 'test_score')

P-value (Anova): < 0.001


In [94]:
dados_combinados = pd.DataFrame()

## Hard

No voting "hard", a predição final é determinada pela maioria dos votos das predições de classe de cada classificador individual. Cada classificador tem igual peso no voto final, e ele simplesmente escolhe a classe que recebeu o maior número de votos dos classificadores individuais como a predição do ensemble.

**Quando Usar:** O voting hard é útil quando você deseja uma decisão final simples e direta baseada na classe mais frequentemente predita pelos modelos. É eficaz quando os modelos são aproximadamente igualmente bons e há uma clara maioria na escolha de classe em muitos casos.

In [95]:
from sklearn.ensemble import VotingClassifier
votacao = VotingClassifier(estimators=[(nome_modelos[0], pipelines[0]),
                                       (nome_modelos[1], pipelines[1]),
                                       (nome_modelos[2], pipelines[2])],
                           voting='hard') # Votação pela maioria
votacao

VotingClassifier(estimators=[('Árvores',
                              Pipeline(steps=[('scaler', StandardScaler()),
                                              ('model',
                                               DecisionTreeClassifier(max_depth=3,
                                                                      random_state=5))])),
                             ('Logística',
                              Pipeline(steps=[('scaler', StandardScaler()),
                                              ('model',
                                               LogisticRegression(random_state=1))])),
                             ('Naive Bayes',
                              Pipeline(steps=[('scaler', StandardScaler()),
                                              ('model', GaussianNB())]))])

In [96]:
# aplicando o votting classifier
validacao = cross_validate(votacao, x_treino, y_treino, cv = skf)
cv_resultados = pd.DataFrame(validacao)
#cv_resultados.sample(5)

cv_resultados['voting'] = 'Hard'
dados_combinados = pd.concat([dados_combinados, cv_resultados])

## Soft

Em algumas situações, você pode querer dar mais peso a um modelo do que a outro. Por exemplo, se um modelo é conhecido por ser mais preciso, você pode querer que o voto desse modelo tenha mais peso na decisão final. Isso é conhecido como `votação ponderada` ou “votação suave”.

In [97]:
from sklearn.ensemble import VotingClassifier
votacao = VotingClassifier(
    estimators=[(nome_modelos[0], pipelines[0]),
                (nome_modelos[1], pipelines[1]),
                (nome_modelos[2], pipelines[2])],
                voting='soft')
votacao

VotingClassifier(estimators=[('Árvores',
                              Pipeline(steps=[('scaler', StandardScaler()),
                                              ('model',
                                               DecisionTreeClassifier(max_depth=3,
                                                                      random_state=5))])),
                             ('Logística',
                              Pipeline(steps=[('scaler', StandardScaler()),
                                              ('model',
                                               LogisticRegression(random_state=1))])),
                             ('Naive Bayes',
                              Pipeline(steps=[('scaler', StandardScaler()),
                                              ('model', GaussianNB())]))],
                 voting='soft')

In [98]:
# validando o votting classifier
validacao = cross_validate(votacao, x_treino, y_treino, cv = skf)
cv_resultados = pd.DataFrame(validacao)
#cv_resultados.sample(5)

cv_resultados['voting'] = 'Soft'
dados_combinados = pd.concat([dados_combinados, cv_resultados])

In [99]:
# Avaliando tipos de votings
num_cat_hip(dados_combinados, 'voting', 'test_score')

P-value (Teste T): 0.79


### Criando grade de peso

In [100]:
parametros = {
    'voting':['hard','soft'],
    'weights':[(1,1,1),(2,1,1),(1,2,1),(1,1,2)]
}

from sklearn.model_selection import GridSearchCV
gridsearch = GridSearchCV(votacao, parametros, n_jobs=-1)
gridsearch.fit(x_treino, y_treino)

GridSearchCV(estimator=VotingClassifier(estimators=[('Árvores',
                                                     Pipeline(steps=[('scaler',
                                                                      StandardScaler()),
                                                                     ('model',
                                                                      DecisionTreeClassifier(max_depth=3,
                                                                                             random_state=5))])),
                                                    ('Logística',
                                                     Pipeline(steps=[('scaler',
                                                                      StandardScaler()),
                                                                     ('model',
                                                                      LogisticRegression(random_state=1))])),
                                                    ('Naive Bayes',
                                                     Pipeline(steps=[('scaler',
                                                                      StandardScaler()),
                                                                     ('model',
                                                                      GaussianNB())]))],
                                        voting='soft'),
             n_jobs=-1,
             param_grid={'voting': ['hard', 'soft'],
                         'weights': [(1, 1, 1), (2, 1, 1), (1, 2, 1),
                                     (1, 1, 2)]})

In [101]:
gridsearch.best_params_

{'voting': 'hard', 'weights': (1, 2, 1)}

In [102]:
gridsearch.best_score_

0.9226666666666666

In [103]:
pd.DataFrame(gridsearch.cv_results_)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_voting,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.031317,0.006484,0.004765,0.000894,hard,"(1, 1, 1)","{'voting': 'hard', 'weights': (1, 1, 1)}",0.893333,0.933333,0.893333,0.940000,0.946667,0.921333,0.023247,2
1,0.027517,0.003828,0.003391,0.000532,hard,"(2, 1, 1)","{'voting': 'hard', 'weights': (2, 1, 1)}",0.866667,0.893333,0.833333,0.846667,0.886667,0.865333,0.022862,8
2,0.021056,0.003307,0.004979,0.001724,hard,"(1, 2, 1)","{'voting': 'hard', 'weights': (1, 2, 1)}",0.900000,0.926667,0.893333,0.940000,0.953333,0.922667,0.022940,1
3,0.021540,0.005494,0.005473,0.002222,hard,"(1, 1, 2)","{'voting': 'hard', 'weights': (1, 1, 2)}",0.900000,0.946667,0.866667,0.913333,0.926667,0.910667,0.026866,5
4,0.020721,0.005243,0.003680,0.002809,soft,"(1, 1, 1)","{'voting': 'soft', 'weights': (1, 1, 1)}",0.900000,0.933333,0.880000,0.933333,0.946667,0.918667,0.024730,4
5,0.020594,0.006341,0.002006,0.000009,soft,"(2, 1, 1)","{'voting': 'soft', 'weights': (2, 1, 1)}",0.886667,0.906667,0.880000,0.926667,0.940000,0.908000,0.022862,7
6,0.018138,0.002323,0.001814,0.000408,soft,"(1, 2, 1)","{'voting': 'soft', 'weights': (1, 2, 1)}",0.906667,0.926667,0.893333,0.933333,0.946667,0.921333,0.019044,2
7,0.015356,0.000864,0.001362,0.000844,soft,"(1, 1, 2)","{'voting': 'soft', 'weights': (1, 1, 2)}",0.886667,0.933333,0.893333,0.920000,0.920000,0.910667,0.017689,5


In [104]:
y_pred = gridsearch.predict(x_teste)
avaliar_modelo(y_teste, y_pred, 'voting')

**********************************************************************
Relatório de Classificação para o Conjunto de voting :

              precision    recall  f1-score   support

           0       0.85      0.94      0.89       124
           1       0.94      0.83      0.88       126

    accuracy                           0.89       250
   macro avg       0.89      0.89      0.89       250
weighted avg       0.89      0.89      0.89       250

**************************************************
Matriz de Confusão para o Conjunto de voting :

[[117   7]
 [ 21 105]]
**********************************************************************


# Hard com pesos vs Soft com pesos

**Voting Hard com Pesos**
No voting "hard" com pesos:
- **Decisão Baseada em Rótulos de Classe**: O classificador toma suas decisões baseado exclusivamente nos rótulos de classe preditos por cada classificador individual.
- **Aplicação de Pesos**: Cada voto (rótulo de classe predito) de um classificador tem um peso associado. Se um classificador com um peso de 2 prediz a classe 1, é como se essa predição contasse como dois votos para a classe 1. O resultado final é determinado pela classe que acumula o maior peso total de votos.
- **Uso**: É útil quando você confia diferentemente na precisão dos modelos individuais e deseja que essa confiança seja refletida na decisão final.

**Voting Soft com Pesos**
No voting "soft" com pesos:
- **Decisão Baseada em Probabilidades**: O classificador leva em consideração a probabilidade prevista de cada classe por cada classificador.
- **Aplicação de Pesos**: As probabilidades previstas são ponderadas conforme os pesos atribuídos a cada classificador. Por exemplo, se um classificador com peso 2 dá uma probabilidade de 0.7 para a classe 1, essa contribuição é dobrada ao calcular a média ponderada das probabilidades.
- **Resultado**: A classe final é escolhida com base na maior média ponderada das probabilidades previstas, não apenas na maioria dos votos.
- **Uso**: É recomendado quando os classificadores estão bem calibrados e suas probabilidades de predição são confiáveis. O modo soft permite que a incerteza nas predições dos classificadores influencie mais diretamente a decisão final.

**Comparação e Implicações**
- **Sensibilidade a Probabilidades**: O voting soft é mais sensível a como os classificadores individuais quantificam sua incerteza nas predições. Ele não apenas conta votos, mas avalia quão seguros os classificadores estão em suas predições.
- **Influência de Classificadores Individuais**: No modo hard, um classificador com alto peso pode influenciar desproporcionalmente o resultado final se sua decisão for contrária à maioria. No modo soft, esse impacto pode ser suavizado se as probabilidades não forem extremamente polarizadas.
- **Flexibilidade e Desempenho**: O voting soft tende a ser mais flexível e potencialmente mais eficaz em capturar nuances nos dados, já que utiliza informações mais ricas (probabilidades em vez de rótulos binários). No entanto, isso pressupõe que as probabilidades de cada classificador sejam bem calibradas e comparáveis.

**Conclusão**
A escolha entre hard e soft voting com pesos deve ser guiada por uma compreensão clara de como cada classificador individual opera, a confiabilidade de suas saídas, e os objetivos específicos de seu problema de classificação. Avaliações experimentais, como validação cruzada, são essenciais para determinar qual abordagem oferece o melhor desempenho em seu contexto específico.

# Bagging - Bootstrap Aggregating

Bagging, ou Bootstrap Aggregating, envolve treinar múltiplos modelos do mesmo tipo em diferentes subconjuntos dos dados de treinamento. Cada modelo é treinado em um subconjunto gerado por amostragem com reposição (bootstrap) do conjunto de dados original. As previsões finais são feitas por votação majoritária (classificação) ou média (regressão).

A diferença no Bagging Classifier está no dataset: ele passará linhas diferentes do dataset para essas árvores, permitindo que essas árvores enxerguem padrões diferentes nos nossos dados.

termo bagging é uma abreviação de `bootstrap aggregating`. Essa é uma técnica de machine learning que visa melhorar a estabilidade e precisão de algoritmos de aprendizagem de máquina. 

Essencialmente, `o bagging envolve a criação de múltiplos conjuntos de dados de treinamento através da amostragem com reposição (conhecida como bootstrap) do conjunto de treinamento original`. 

`Modelos separados são treinados de forma independente em cada um desses conjuntos de dados boostrap. Após o treinamento, as previsões desses múltiplos modelos são combinadas`. 

Geralmente o processo de combinação é feito pela votação para a classificação ou média para regressão. Isso é benéfico principalmente para algoritmos que são sensíveis a variações nos dados de treinamento, como árvores de decisão.

- **Vantagens**:
  - Reduz o overfitting e aumenta a estabilidade do modelo.
  - Melhora a precisão de modelos que têm alta variância.
  
- **Desvantagens**:
  - Pode ser computacionalmente intensivo, pois requer treinamento de múltiplos modelos.
  - Menos eficaz se os modelos individuais são muito enviesados.

In [105]:
from sklearn.ensemble import BaggingClassifier

bagging_classifier = BaggingClassifier(n_estimators=10, random_state=42)
bagging_classifier.fit(x_treino, y_treino)

BaggingClassifier(random_state=42)

In [106]:
y_pred = bagging_classifier.predict(x_teste)
avaliar_modelo(y_pred, y_teste, 'Bagging Classifier')

**********************************************************************
Relatório de Classificação para o Conjunto de Bagging Classifier :

              precision    recall  f1-score   support

           0       0.93      0.82      0.87       140
           1       0.80      0.92      0.86       110

    accuracy                           0.86       250
   macro avg       0.86      0.87      0.86       250
weighted avg       0.87      0.86      0.86       250

**************************************************
Matriz de Confusão para o Conjunto de Bagging Classifier :

[[115  25]
 [  9 101]]
**********************************************************************


In [107]:
# Validando Begging

# Definir os modelos base # instanciando modelo
modelo_base = pipelines[0] 

# Definir os parâmetros para o GridSearchCV
parametros_grid = {
    'n_estimators': [10, 20, 30],
    'max_samples': [0.5, 0.7, 0.9],
    'max_features': [0.5, 0.7, 0.9],
}

# Criar o BaggingClassifier com o GridSearchCV
bagging_grid = GridSearchCV(
    BaggingClassifier(),
    parametros_grid,
    cv=skf,
    n_jobs=-1
)

bagging_grid.fit(x_treino, y_treino) # Treinar o modelo final

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=5, shuffle=True),
             estimator=BaggingClassifier(), n_jobs=-1,
             param_grid={'max_features': [0.5, 0.7, 0.9],
                         'max_samples': [0.5, 0.7, 0.9],
                         'n_estimators': [10, 20, 30]})

In [108]:
# Obter a melhor combinação de parâmetros encontrada pelo GridSearchCV
melhores_parametros = bagging_grid.best_params_
print(melhores_parametros)

print(bagging_grid.best_score_)

{'max_features': 0.7, 'max_samples': 0.5, 'n_estimators': 30}
0.932


In [109]:
# Criar o BaggingClassifier com os melhores parâmetros encontrados
bagging_classifier = BaggingClassifier(estimator=modelo_base, **melhores_parametros)
bagging_classifier.fit(x_treino, y_treino) # Treinar o modelo final

# Fazer previsões
y_pred = bagging_classifier.predict(x_teste)
avaliar_modelo(y_pred, y_teste, 'BaggingClassifier')

**********************************************************************
Relatório de Classificação para o Conjunto de BaggingClassifier :

              precision    recall  f1-score   support

           0       0.90      0.82      0.85       136
           1       0.80      0.89      0.84       114

    accuracy                           0.85       250
   macro avg       0.85      0.85      0.85       250
weighted avg       0.85      0.85      0.85       250

**************************************************
Matriz de Confusão para o Conjunto de BaggingClassifier :

[[111  25]
 [ 13 101]]
**********************************************************************


# ExtraTreesClassifier - Extremely Randomized Trees Classifier

É um algoritmo de ensemble baseado em árvores de decisão que faz parte da família de métodos de "bagging". Ele é uma variação do Random Forest e pertence à classe de algoritmos que constroem múltiplas árvores de decisão para realizar previsões agregadas, seja para problemas de classificação ou regressão. 

O **ExtraTreesClassifier** é similar ao **Random Forest**, mas com algumas diferenças importantes que tornam o algoritmo ainda mais aleatório, o que pode melhorar o desempenho em alguns casos.

**Treinamento de Árvores**

Como no Random Forest, o **ExtraTreesClassifier** constrói várias árvores de decisão, onde cada árvore é treinada com um subconjunto de dados. No entanto, no ExtraTrees, **não há amostragem com reposição dos dados** (bootstrap). Ou seja, ele utiliza o conjunto de dados original em vez de amostras bootstrapped.

**Aleatoriedade no Corte dos Atributos**

- **Random Forest**: Em cada nó da árvore, o Random Forest seleciona aleatoriamente um subconjunto de características e, para cada característica desse subconjunto, determina o melhor ponto de divisão (*threshold*) para separar os dados.

- **ExtraTrees**: O **ExtraTreesClassifier** vai um passo além na aleatoriedade. Ele não apenas seleciona aleatoriamente um subconjunto de características, mas também escolhe **aleatoriamente o ponto de divisão** dentro do conjunto de possíveis divisões, em vez de escolher o ponto que melhor separa os dados. Isso adiciona mais variabilidade às árvores, o que pode reduzir o overfitting, mas, ao mesmo tempo, aumenta a aleatoriedade.

**Agregação**

- Para tarefas de **classificação**, a predição final é feita com base na votação majoritária das predições de todas as árvores.
- Para tarefas de **regressão**, a predição final é a média das previsões de todas as árvores.

**Diferenças Chave entre ExtraTrees e Random Forest**

- **Divisões Aleatórias**: Enquanto o Random Forest escolhe a melhor divisão para uma determinada característica, o ExtraTrees escolhe a divisão de forma completamente aleatória. Isso resulta em árvores ainda mais diferentes entre si, o que pode levar a uma maior redução da variância, mas com o custo de potencialmente aumentar o viés.

**Vantagens do ExtraTreesClassifier**

- **Menor Variância**: Devido à alta aleatoriedade na construção das árvores, o ExtraTrees tende a produzir um modelo com menor variância em comparação com uma única árvore de decisão, e até mesmo comparado ao Random Forest, que já usa a aleatoriedade.

- **Maior Velocidade**: Como o ExtraTrees não precisa calcular o ponto ideal de divisão em cada nó (como no Random Forest), ele pode ser mais rápido para treinar. A escolha aleatória do ponto de divisão simplifica o processo de construção da árvore.

- **Redução de Overfitting**: A aleatoriedade adicional pode ajudar a prevenir overfitting, especialmente em dados ruidosos.

**Desvantagens do ExtraTreesClassifier**

- **Aumento de Viés**: A desvantagem da aleatoriedade extra é que ela pode aumentar o viés do modelo




In [110]:
from sklearn.ensemble import ExtraTreesClassifier

parametros_grid = {
    'n_estimators': [10, 20, 30],
    'max_features': [0.5, 0.7, 0.9]
}

extratrees_grid = GridSearchCV(
    ExtraTreesClassifier(),
    parametros_grid,
    cv=skf, 
    n_jobs=-1
)

# Treinar e ajustar o modelo
extratrees_grid.fit(x_treino, y_treino)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=5, shuffle=True),
             estimator=ExtraTreesClassifier(), n_jobs=-1,
             param_grid={'max_features': [0.5, 0.7, 0.9],
                         'n_estimators': [10, 20, 30]})

In [111]:
# Obter a melhor combinação de parâmetros encontrada pelo GridSearchCV
melhores_parametros = extratrees_grid.best_params_
print(melhores_parametros)

print(extratrees_grid.best_score_)

{'max_features': 0.5, 'n_estimators': 20}
0.8733333333333333


In [112]:
# Criar o ExtraTreesClassifier com os melhores parâmetros encontrados
extratrees_classifier = ExtraTreesClassifier(**melhores_parametros)
extratrees_classifier.fit(x_treino, y_treino) # Treinar o modelo final

# Fazer previsões
y_pred = bagging_classifier.predict(x_teste)
avaliar_modelo(y_pred, y_teste, 'ExtraTreesClassifier')

**********************************************************************
Relatório de Classificação para o Conjunto de ExtraTreesClassifier :

              precision    recall  f1-score   support

           0       0.90      0.82      0.85       136
           1       0.80      0.89      0.84       114

    accuracy                           0.85       250
   macro avg       0.85      0.85      0.85       250
weighted avg       0.85      0.85      0.85       250

**************************************************
Matriz de Confusão para o Conjunto de ExtraTreesClassifier :

[[111  25]
 [ 13 101]]
**********************************************************************


## vs
O **ExtraTreesClassifier**, também conhecido como **Extremely Randomized Trees**, é um tipo de ensemble learning que faz parte da família das árvores de decisão. Ele tem algumas diferenças fundamentais em relação ao **BaggingClassifier** quando este é usado com árvores de decisão.

Durante a construção das árvores, o ExtraTrees seleciona aleatoriamente um subconjunto de colunas e, para cada recurso, escolhe um ponto de divisão aleatório. Isto é, ele usa limites aleatórios para cada recurso. Já no **BaggingClassifier**, cada árvore tenta encontrar o melhor ponto de divisão.

No ExtraTrees, as árvores são construídas usando todas as linhas do dataset de treinamento (embora possam ser usadas amostras bootstrap como opção). No **Bagging**, cada árvore recebe uma amostra de linhas diferentes do dataset de treinamento.

**Decision Tree**
A **Decision Tree** (Árvore de Decisão) é um modelo de aprendizado supervisionado que funciona dividindo repetidamente os dados com base em regras de decisão derivadas dos recursos (features). As características principais de uma árvore de decisão são:

- **Divisões Otimizadas**: Cada nó da árvore tenta encontrar a melhor divisão (o melhor ponto de separação) para o conjunto de dados naquele momento.
- **Simples e Interpretável**: A árvore de decisão pode ser facilmente visualizada e interpretada. Cada caminho da raiz até uma folha representa uma sequência de decisões lógicas que levam a uma predição.
- **Problemas com Overfitting**: Árvores de decisão são propensas a overfitting, especialmente quando são profundas, já que podem capturar detalhes e ruídos específicos dos dados de treinamento.

**Random Forest**
O **Random Forest** é um ensemble de árvores de decisão, onde cada árvore é construída a partir de um subconjunto dos dados de treinamento, gerado por amostragem com reposição (bootstrap), e com um subconjunto aleatório de recursos (features) em cada nó.

- **Bagging e Subconjuntos de Features**: O Random Forest aplica a técnica de **bagging** (bootstrap aggregating) para criar diferentes amostras dos dados de treinamento para cada árvore. Além disso, em cada nó da árvore, é selecionado aleatoriamente um subconjunto de features para encontrar o melhor ponto de divisão.
- **Redução de Variância**: Ao combinar os resultados de várias árvores (por meio de votação para classificação ou média para regressão), o Random Forest reduz a variância e o risco de overfitting, tornando-o mais robusto do que uma única árvore de decisão.
- **Eficiência**: O Random Forest é eficiente e bastante preciso em muitas tarefas de classificação e regressão, pois suas árvores são relativamente independentes e se beneficiam da combinação de suas previsões.

**Bagging (Bootstrap Aggregating)**
O **BaggingClassifier** é uma técnica de ensemble que usa o princípio do bagging, ou **bootstrap aggregating**, para treinar múltiplos modelos de forma independente em diferentes amostras bootstrap dos dados de treinamento.

- **Amostragem com Reposição (Bootstrap)**: No bagging, cada modelo recebe uma amostra aleatória com reposição do conjunto de dados original. Isso cria várias versões dos dados de treinamento, cada uma com possíveis repetições de amostras.
- **Modelos Independentes**: Cada modelo base (geralmente árvores de decisão) é treinado de forma independente. As previsões dos modelos são combinadas por votação (para classificação) ou por média (para regressão).
- **Redução de Variância**: O bagging ajuda a reduzir a variância, tornando os modelos mais robustos e resistentes ao overfitting. No entanto, a aleatoriedade adicional nas amostras melhora a estabilidade do modelo sem alterar significativamente seu viés.

**ExtraTreesClassifier**
Como mencionado, o **ExtraTreesClassifier** é um método semelhante ao Random Forest, mas com mais aleatoriedade no processo de construção das árvores. As diferenças principais incluem:

- **Divisões Aleatórias**: No ExtraTrees, o ponto de divisão em cada nó é escolhido de forma aleatória, dentro de um subconjunto de recursos, em vez de tentar encontrar o ponto de divisão ideal. Isso adiciona mais variabilidade às árvores.
- **Uso Completo dos Dados**: Ao contrário do Random Forest e do Bagging, o ExtraTrees não usa amostragem bootstrap por padrão. Em vez disso, ele utiliza todo o conjunto de dados original para construir cada árvore.
- **Menor Variância**: Devido à sua maior aleatoriedade, o ExtraTrees tende a ter menor variância e, em muitos casos, pode ser mais rápido para treinar, pois não precisa encontrar a melhor divisão para cada nó.
- **Potencial Aumento de Viés**: O custo dessa maior aleatoriedade é um potencial aumento de viés, o que pode resultar em menor precisão, especialmente em problemas onde uma boa divisão é crítica para o desempenho.

**Conclusão**

- **Decision Tree**: Modelo simples e interpretável que busca a melhor divisão em cada nó, mas é propenso ao overfitting.
- **Random Forest**: Ensemble de árvores de decisão construídas com amostras bootstrap e subconjuntos aleatórios de features, que reduz a variância e melhora a robustez.
- **Bagging**: Técnica de ensemble que treina modelos independentes em amostras bootstrap e combina suas previsões para melhorar a estabilidade.
- **ExtraTreesClassifier**: Similar ao Random Forest, mas com maior aleatoriedade no ponto de divisão dos nós e sem amostragem bootstrap por padrão. Oferece menor variância, mas pode aumentar o viés.

Esses modelos formam a base de várias técnicas avançadas de aprendizado de máquina e podem ser aplicados a uma ampla gama de problemas de classificação e regressão, cada um com suas vantagens e desvantagens dependendo do cenário.


# -----------------------------------------------------------------

O método de voting possibilitou combinar diferentes modelos para aperfeiçoar nosso modelo de classificação, e o método de bagging permitiu aplicar diferentes versões do nosso conjunto de dados a modelos de árvores, com o objetivo de também melhorar nosso modelo de classificação.

# Boosting

## AdaBoost

Esse modelo opera com modelos de árvores de maneira sequencial, aperfeiçoando o funcionamento dos modelos de árvores a cada iteração. Temos uma taxa de aprendizado (learning_rate) que pode variar. O n_estimators representará a quantidade de árvores que passaremos dentro do AdaBoostClassifier.

O AdaBoost (Adaptive Boosting) é um algoritmo de aprendizado de máquina baseado na combinação de modelos. Ele começa treinando um classificador (geralmente uma árvore de decisão) no dataset original. Em seguida, treina iterativamente uma série de classificadores adicionais, cada um focando nos casos onde o classificador anterior falhou. Em cada rodada, o AdaBoost ajusta os pesos das instâncias de treinamento aumentando os pesos dos casos mal classificados, para que o próximo classificador se concentre mais neles, e diminuindo os pesos dos casos bem classificados. Os classificadores são então ponderados de acordo com sua precisão e combinados para formar o modelo final, onde as decisões são levadas em conta de acordo com seu desempenho, resultando em uma maior capacidade de generalização e robustez em comparação a um classificador único.

In [113]:
from sklearn.ensemble import AdaBoostClassifier

modelo = AdaBoostClassifier(n_estimators=50, #numero de arvores
                            learning_rate=1
                            )
modelo.fit(x_treino, y_treino)

AdaBoostClassifier(learning_rate=1)

In [114]:
y_pred = modelo.predict(x_teste)
avaliar_modelo(y_pred, y_teste, 'AdaBoostClassifier')

**********************************************************************
Relatório de Classificação para o Conjunto de AdaBoostClassifier :

              precision    recall  f1-score   support

           0       0.93      0.86      0.89       134
           1       0.85      0.92      0.88       116

    accuracy                           0.89       250
   macro avg       0.89      0.89      0.89       250
weighted avg       0.89      0.89      0.89       250

**************************************************
Matriz de Confusão para o Conjunto de AdaBoostClassifier :

[[115  19]
 [  9 107]]
**********************************************************************


In [115]:
# Buscando os melhores parametros
modelo = AdaBoostClassifier()

param_grid = {
'n_estimators': [50,100,200],
'learning_rate': [0.1,0.01,0.001]
}

grid_search = GridSearchCV(modelo, param_grid, cv=skf, scoring='accuracy',n_jobs=-1)
grid_search.fit(x_treino,y_treino)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=5, shuffle=True),
             estimator=AdaBoostClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.01, 0.001],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy')

In [116]:
y_pred = grid_search.predict(x_teste)
avaliar_modelo(y_pred, y_teste, 'AdaBoost')

**********************************************************************
Relatório de Classificação para o Conjunto de AdaBoost :

              precision    recall  f1-score   support

           0       0.91      0.86      0.89       131
           1       0.86      0.91      0.88       119

    accuracy                           0.88       250
   macro avg       0.88      0.89      0.88       250
weighted avg       0.89      0.88      0.88       250

**************************************************
Matriz de Confusão para o Conjunto de AdaBoost :

[[113  18]
 [ 11 108]]
**********************************************************************


# CatBoost

O CatBoost, abreviação de Category Boosting, é um algoritmo de boosting baseado em árvores de decisão, desenvolvido pela equipe da Yandex. Como um membro da família dos algoritmos de gradient boosting, o Catboost se destaca por sua habilidade de lidar com dados categóricos sem a necessidade de extenso pré-processamento. Isso é possível graças ao seu método inovador de codificação que melhora a eficiência e reduz o risco de overfitting.

No tratamento de dados categóricos o CatBoost lida de maneira nativa com esses dados, algo que outros algoritmos de boosting não fazem diretamente. Isso elimina a necessidade de pré-processamento como one-hot enconding, poupando tempo e evitando a expansão dimensional que pode acompanhar essas técnicas. Além disso, através de técnicas de regularização e uma estratégia de gradiente otimizada, o CatBoost se torna particularmente robusto contra o overfitting.

**Funcionamento -**
O CatBoost opera em um processo iterativo onde constrói árvore por árvore. Cada árvore tenta corrigir os erros da árvore anterior, focando nos casos mais difíceis. Porém, ao contrário de outros algoritmos que ajustam os pesos dos dados, o CatBoost ajusta o valor do gradiente para cada árvore, melhorando a precisão do modelo de maneira incremental.

Esse método tem uma abordagem otimizada para dados categóricos e resistência ao overfitting, ele não só melhora a precisão das previsões, mas também simplifica o fluxo de trabalho de pré-processamento de dados.

In [117]:
from catboost import CatBoostClassifier

param_grid = {
'iterations': [100, 200,300],
'depth': [4,6,8],
'learning_rate': [0.1,0.01,0.001]
}

grid_search = GridSearchCV(estimator=CatBoostClassifier(verbose=0), # verbose é para não mostrar warning e outras saidas do algoritmo
                           param_grid = param_grid, 
                           cv=5,
                           scoring='accuracy',
                           n_jobs=-1)
grid_search.fit(x_treino, y_treino)

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostClassifier object at 0x0000021E989D5010>,
             n_jobs=-1,
             param_grid={'depth': [4, 6, 8], 'iterations': [100, 200, 300],
                         'learning_rate': [0.1, 0.01, 0.001]},
             scoring='accuracy')

In [118]:
# Obter a melhor combinação de parâmetros encontrada pelo GridSearchCV
melhores_parametros = grid_search.best_params_
print(melhores_parametros)

print(grid_search.best_score_)

{'depth': 4, 'iterations': 200, 'learning_rate': 0.1}
0.9613333333333334


In [119]:
y_pred = grid_search.predict(x_teste)
avaliar_modelo(y_pred, y_teste, 'AdaBoost')

**********************************************************************
Relatório de Classificação para o Conjunto de AdaBoost :

              precision    recall  f1-score   support

           0       0.98      0.93      0.95       130
           1       0.93      0.97      0.95       120

    accuracy                           0.95       250
   macro avg       0.95      0.95      0.95       250
weighted avg       0.95      0.95      0.95       250

**************************************************
Matriz de Confusão para o Conjunto de AdaBoost :

[[121   9]
 [  3 117]]
**********************************************************************


**WOOWW, ESSE FOI O MELHOR MODELO ATÉ AGORA**

In [120]:
from catboost import CatBoostClassifier

param_grid = {
'iterations': [100, 200,300],
'depth': [4,6,8],
'learning_rate': [0.1,0.01,0.001]
}

grid_search = GridSearchCV(estimator=CatBoostClassifier(verbose=0), # verbose é para não mostrar warning e outras saidas do algoritmo
                           param_grid = param_grid, 
                           cv=skf,
                           scoring='accuracy',
                           n_jobs=-1)
grid_search.fit(x_treino, y_treino)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=5, shuffle=True),
             estimator=<catboost.core.CatBoostClassifier object at 0x0000021E989DFAD0>,
             n_jobs=-1,
             param_grid={'depth': [4, 6, 8], 'iterations': [100, 200, 300],
                         'learning_rate': [0.1, 0.01, 0.001]},
             scoring='accuracy')

In [121]:
# Obter a melhor combinação de parâmetros encontrada pelo GridSearchCV
melhores_parametros = grid_search.best_params_
print(melhores_parametros)

print(grid_search.best_score_)

{'depth': 6, 'iterations': 300, 'learning_rate': 0.1}
0.9600000000000002


In [122]:
y_pred = grid_search.predict(x_teste)
avaliar_modelo(y_pred, y_teste, 'AdaBoost')

**********************************************************************
Relatório de Classificação para o Conjunto de AdaBoost :

              precision    recall  f1-score   support

           0       0.96      0.92      0.94       129
           1       0.92      0.96      0.94       121

    accuracy                           0.94       250
   macro avg       0.94      0.94      0.94       250
weighted avg       0.94      0.94      0.94       250

**************************************************
Matriz de Confusão para o Conjunto de AdaBoost :

[[119  10]
 [  5 116]]
**********************************************************************


# Stacking

É um metodo semelhante ao voting, possibilita combinar tantos modelos simples e modelos complexos.

O StackingClassifier à primeira vista parece bastante com o método de votação. Porém, essa técnica utiliza um meta-classificador na combinação do resultado de uma série de modelos de base. Primeiro, um conjunto de modelos de aprendizado de máquina, conhecidos como modelos base ou de nível 0, são treinados independentemente nos mesmos dados de treino. Estes modelos podem ser de tipos variados, como árvores de decisão, regressão logística, SVMs, etc., e cada um fará suas próprias previsões. As previsões dos modelos base são então usadas como entrada para criar um novo conjunto de treinamento. Este conjunto é construído de tal forma que as saídas dos modelos base tornam-se características que alimentam o próximo nível do stacking.

Em seguida, um meta-modelo, também conhecido como modelo de nível 1, é treinado neste novo conjunto de dados. O trabalho do meta-modelo é aprender como combinar melhor as previsões dos modelos base para fazer uma previsão final. O meta-modelo pode ser um modelo diferente dos modelos base ou até mesmo o mesmo tipo, mas é treinado para focar em como integrar as informações dos modelos base ao invés de aprender diretamente dos dados originais.

O que torna o stacking tão poderoso é que ele pode capturar os pontos fortes de cada modelo base e, ao mesmo tempo, compensar suas fraquezas, resultando em um modelo composto que é frequentemente mais preciso e robusto do que qualquer um dos modelos individuais. Porém, é importante notar que o stacking pode ser computacionalmente intensivo e corre o risco de overfitting, especialmente se o número de modelos base for grande ou se os modelos forem altamente correlacionados. Portanto, uma seleção cuidadosa dos modelos base e do meta-modelo é essencial para o sucesso desta técnica.

## Combinação de modelos simples

In [123]:
from sklearn.ensemble import StackingClassifier

# Instanciando modelos
modelos_base = [
    (nome_modelos[0], pipelines[0]),
    (nome_modelos[1], pipelines[1]),
    (nome_modelos[2], pipelines[2])
    ]
modelos_base

[('Árvores',
  Pipeline(steps=[('scaler', StandardScaler()),
                  ('model', DecisionTreeClassifier(max_depth=3, random_state=5))])),
 ('Logística',
  Pipeline(steps=[('scaler', StandardScaler()),
                  ('model', LogisticRegression(random_state=1))])),
 ('Naive Bayes',
  Pipeline(steps=[('scaler', StandardScaler()), ('model', GaussianNB())]))]

In [124]:
modelo_meta = LogisticRegression(random_state=RANDOM_STATE)
modelo_meta

LogisticRegression(random_state=3141592)

Por que temos esse modelo extra, que estamos definindo como modelo_meta? Porque em vez de ter um processo simples de votação, teremos um resultado de classificação com base nesses modelos-base, e modelo_meta aprenderá a fazer o que este conjunto de modelos-base faz.

Ele vai se ajustar aos resultados da combinação desses três modelos, o que tende a funcionar melhor com esse método, o StackingClassifier.

In [125]:
stacking_classifier = StackingClassifier(estimators=modelos_base,
                                         final_estimator=modelo_meta)
stacking_classifier

StackingClassifier(estimators=[('Árvores',
                                Pipeline(steps=[('scaler', StandardScaler()),
                                                ('model',
                                                 DecisionTreeClassifier(max_depth=3,
                                                                        random_state=5))])),
                               ('Logística',
                                Pipeline(steps=[('scaler', StandardScaler()),
                                                ('model',
                                                 LogisticRegression(random_state=1))])),
                               ('Naive Bayes',
                                Pipeline(steps=[('scaler', StandardScaler()),
                                                ('model', GaussianNB())]))],
                   final_estimator=LogisticRegression(random_state=3141592))

Temos uma estrutura parecida com o voting, com os três modelos combinados em um único resultado, mas esse único resultado é pego por um estimador final, mostrado na figura acima, e esta figura mostra que o modelo combina, entende, aprende com o resultado final dado pelos três modelos.

In [126]:
stacking_classifier.fit(x_treino, y_treino)

StackingClassifier(estimators=[('Árvores',
                                Pipeline(steps=[('scaler', StandardScaler()),
                                                ('model',
                                                 DecisionTreeClassifier(max_depth=3,
                                                                        random_state=5))])),
                               ('Logística',
                                Pipeline(steps=[('scaler', StandardScaler()),
                                                ('model',
                                                 LogisticRegression(random_state=1))])),
                               ('Naive Bayes',
                                Pipeline(steps=[('scaler', StandardScaler()),
                                                ('model', GaussianNB())]))],
                   final_estimator=LogisticRegression(random_state=3141592))

In [127]:
y_pred = stacking_classifier.predict(x_teste)
avaliar_modelo(y_pred, y_teste, 'Stacking')

**********************************************************************
Relatório de Classificação para o Conjunto de Stacking :

              precision    recall  f1-score   support

           0       0.93      0.87      0.90       132
           1       0.87      0.92      0.89       118

    accuracy                           0.90       250
   macro avg       0.90      0.90      0.90       250
weighted avg       0.90      0.90      0.90       250

**************************************************
Matriz de Confusão para o Conjunto de Stacking :

[[115  17]
 [  9 109]]
**********************************************************************


## Combinação de modelos complexo e fazendo validação

In [128]:
from mlxtend.classifier import StackingCVClassifier

#Melhores parâmetros dos modelos com o gridsearch que fizemos lá encima
parametros_cat = grid_search.best_params_
parametros_trees = extratrees_grid.best_params_
base_models = [
    CatBoostClassifier(**parametros_cat),
    ExtraTreesClassifier(**parametros_trees),
    pipelines[2]
]

# **combinação**
stacking_cv_classifier = StackingCVClassifier(classifiers=base_models, meta_classifier=modelo_meta)
stacking_cv_classifier.fit(x_treino, y_treino)

0:	learn: 0.6031908	total: 6.07ms	remaining: 1.81s
1:	learn: 0.5359041	total: 10.9ms	remaining: 1.63s
2:	learn: 0.4797160	total: 15.7ms	remaining: 1.55s
3:	learn: 0.4401931	total: 21.4ms	remaining: 1.58s
4:	learn: 0.4059752	total: 25.3ms	remaining: 1.49s
5:	learn: 0.3729783	total: 29.3ms	remaining: 1.43s
6:	learn: 0.3405018	total: 34.3ms	remaining: 1.44s
7:	learn: 0.3135445	total: 37.9ms	remaining: 1.38s
8:	learn: 0.2861820	total: 41.3ms	remaining: 1.34s
9:	learn: 0.2768254	total: 44ms	remaining: 1.27s
10:	learn: 0.2604408	total: 49.4ms	remaining: 1.3s
11:	learn: 0.2447036	total: 53ms	remaining: 1.27s
12:	learn: 0.2351291	total: 56.6ms	remaining: 1.25s
13:	learn: 0.2214369	total: 59.8ms	remaining: 1.22s
14:	learn: 0.2116020	total: 64.5ms	remaining: 1.22s
15:	learn: 0.2008188	total: 67.7ms	remaining: 1.2s
16:	learn: 0.1904433	total: 71ms	remaining: 1.18s
17:	learn: 0.1812749	total: 74ms	remaining: 1.16s
18:	learn: 0.1708481	total: 76.6ms	remaining: 1.13s
19:	learn: 0.1631351	total: 81.3

StackingCVClassifier(classifiers=[<catboost.core.CatBoostClassifier object at 0x0000021E988E6210>,
                                  ExtraTreesClassifier(max_features=0.5,
                                                       n_estimators=20),
                                  Pipeline(steps=[('scaler', StandardScaler()),
                                                  ('model', GaussianNB())])],
                     meta_classifier=LogisticRegression(random_state=3141592))

In [132]:
y_pred = stacking_cv_classifier.predict(x_teste)
avaliar_modelo(y_pred, y_teste, 'stacking_cv_classifier')

**********************************************************************
Relatório de Classificação para o Conjunto de stacking_cv_classifier :

              precision    recall  f1-score   support

           0       0.96      0.92      0.94       129
           1       0.92      0.96      0.94       121

    accuracy                           0.94       250
   macro avg       0.94      0.94      0.94       250
weighted avg       0.94      0.94      0.94       250

**************************************************
Matriz de Confusão para o Conjunto de stacking_cv_classifier :

[[119  10]
 [  5 116]]
**********************************************************************


In [137]:
df = pd.DataFrame()

# fazendo Validação do modelo
validacao = cross_validate(stacking_cv_classifier, x_treino, y_treino, 
                           cv=5, n_jobs=-1)
cv_resultados = pd.DataFrame(validacao)
cv_resultados['Modelo'] = 'stacking_cv_classifier\nKfolds'
df = pd.concat([df, cv_resultados])

validacao = cross_validate(stacking_cv_classifier, x_treino, y_treino, 
                           cv = skf, n_jobs=-1)
cv_resultados = pd.DataFrame(validacao)
cv_resultados['Modelo'] = 'stacking_cv_classifie\nStratifiedKFold'
df = pd.concat([df, cv_resultados])

num_cat_hip(df, 'Modelo', 'test_score')

0:	learn: 0.6006511	total: 2.81ms	remaining: 842ms
1:	learn: 0.5367489	total: 5.22ms	remaining: 777ms
2:	learn: 0.4760992	total: 7.38ms	remaining: 731ms
3:	learn: 0.4330867	total: 9.47ms	remaining: 701ms
4:	learn: 0.3900741	total: 14.5ms	remaining: 853ms
5:	learn: 0.3549888	total: 16.8ms	remaining: 821ms
6:	learn: 0.3230215	total: 18.9ms	remaining: 791ms
7:	learn: 0.2945922	total: 21.1ms	remaining: 769ms
8:	learn: 0.2696335	total: 23.1ms	remaining: 747ms
9:	learn: 0.2578649	total: 24.2ms	remaining: 703ms
10:	learn: 0.2431598	total: 26.6ms	remaining: 699ms
11:	learn: 0.2276555	total: 29.6ms	remaining: 710ms
12:	learn: 0.2132094	total: 31.7ms	remaining: 700ms
13:	learn: 0.2013103	total: 33.7ms	remaining: 689ms
14:	learn: 0.1904175	total: 35.8ms	remaining: 680ms
15:	learn: 0.1786226	total: 37.9ms	remaining: 673ms
16:	learn: 0.1679533	total: 40ms	remaining: 667ms
17:	learn: 0.1592230	total: 42.9ms	remaining: 673ms
18:	learn: 0.1499869	total: 48.2ms	remaining: 712ms
19:	learn: 0.1417358	tot

### Salvando modelo

In [143]:
import joblib

joblib.dump(stacking_cv_classifier, 'modelo_treinado.pkl')

['modelo_treinado.pkl']

In [144]:
modelo_carregado = joblib.load('modelo_treinado.pkl')
modelo_carregado

StackingCVClassifier(classifiers=[<catboost.core.CatBoostClassifier object at 0x0000021E99B97090>,
                                  ExtraTreesClassifier(max_features=0.5,
                                                       n_estimators=20),
                                  Pipeline(steps=[('scaler', StandardScaler()),
                                                  ('model', GaussianNB())])],
                     meta_classifier=LogisticRegression(random_state=3141592))

In [158]:
modelo_carregado.predict(x_teste)

array([0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 1, 1])

# FIM